In [1]:
import chipwhisperer as cw
import matplotlib.pyplot as plt

n = 4

try:
    scope.dis()
    target.dis()
except Exception as e:
    pass

scope = cw.scope()
target = cw.target(scope)

scope.default_setup()

cw.program_target(scope, cw.programmers.STM32FProgrammer, "./recall-CWNANO-true.hex")

scope.arm()

# scope.adc.samples = 800
scope.adc.samples = 1000


See https://chipwhisperer.readthedocs.io/en/latest/api.html#firmware-update


Detected known STMF32: STM32F04xxx
Extended erase (0x44), this can take ten seconds or more
Attempting to program 4983 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 4983 bytes


In [2]:
import time
def reset_target(scope): 
    scope.io.nrst = 'low'
    time.sleep(0.01)
    scope.io.nrst = 'high'
    time.sleep(0.01)

def cap_pass_trace(pass_guess):
    ret = ""
#     reset_target(scope)
    num_char = target.in_waiting()
    while num_char > 0:
        ret += target.read(num_char, 16)
        time.sleep(0.01)
        num_char = target.in_waiting()

    scope.arm()
#     target.write(pass_guess)
    target.simpleserial_write('a', pass_guess)

    ret = scope.capture()
    if ret:
        print('Timeout happened during acquisition')

    trace = scope.get_last_trace()
    return trace

In [3]:
import numpy as np

found_elts = []

for pos in range(16):
    ref_msg = bytearray(found_elts)  + bytearray([255]*(16-len(found_elts)))
    ref_trace = cap_pass_trace(ref_msg)
    
    
    temp =[]
    for i in range(0,256):
        msg = bytearray(found_elts) + bytearray([i]) + bytearray([255] * (15-len(found_elts)) )

        trace = cap_pass_trace(msg)
#         diff = int(round(np.sum(np.abs(trace - ref_trace)), -1))
        diff = int(round(np.sum(np.abs(trace - ref_trace))))
        temp.append(diff)
#         print(f"pos: {pos}, i: {i}, diff: {diff}")
        
    temp = np.array(temp)
    mean = temp.mean()
    temp = abs(temp - mean)
    max_elt = temp.argmax()
        
    print(f"position: {pos}, found_elt: {max_elt}")
    found_elts.append(max_elt)


position: 0, found_elt: 112
position: 1, found_elt: 48
position: 2, found_elt: 119
position: 3, found_elt: 51
position: 4, found_elt: 82
position: 5, found_elt: 49
position: 6, found_elt: 115
position: 7, found_elt: 107
position: 8, found_elt: 110
position: 9, found_elt: 48
position: 10, found_elt: 119
position: 11, found_elt: 108
position: 12, found_elt: 51
position: 13, found_elt: 100
position: 14, found_elt: 103
position: 15, found_elt: 51


In [4]:
print(''.join([chr(i) for i in found_elts]))

p0w3R1skn0wl3dg3


In [6]:
import matplotlib.pyplot as plt

In [7]:
found_elts = []

for pos in range(1):
    ref_msg = bytearray(found_elts)  + bytearray([255]*(16-len(found_elts)))
    ref_trace = cap_pass_trace(ref_msg)
    
    
    temp =[]
    traces = []
    for i in range(0,256):
        msg = bytearray(found_elts) + bytearray([i]) + bytearray([255] * (15-len(found_elts)) )
        trace = cap_pass_trace(msg)
        traces.append(trace)
        diff = int(round(np.sum(np.abs(trace - ref_trace))))
        temp.append(diff)
        
    temp = np.array(temp)
    mean = temp.mean()
    temp = abs(temp - mean)
    max_elt = temp.argmax()
        
    print(f"position: {pos}, found_elt: {max_elt}")
    found_elts.append(max_elt)

position: 0, found_elt: 112


In [19]:
%matplotlib tk
plot1, = plt.plot(ref_trace, label='reference')
# plot2, = plt.plot(traces[111],alpha=0.5,label='wrong-guess')
plot3, = plt.plot(traces[112],alpha=0.5,label='wrong-guess')
plt.legend([plot1,plot3],["reference", "correct-guess"])
plt.show()